In [ ]:
%set_env LLAMA_LOG_VERBOSITY=4

# 42 test

- [TinyStories-656K](https://hf-mirror.com/mradermacher/TinyStories-656K-GGUF)

<details>
<summary>config</summary>

```json
{
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 384,
  "max_position_embeddings": 512,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 8,
  "num_hidden_layers": 2,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "vocab_size": 2048
}
```
</details>

In [ ]:
model = "models/TinyStories-656K.f16.gguf"
assert os.path.exists(model)
!llama.cpp/bin/llama-cli -m {model} -p "the answer to the ultimate question of life, the universe, and everything is 42. One day, they " \
-t 1 -n 2 --temp 0 --top-k 0 --top-p 1.0 --min-p 0.0 2>/dev/null

# Hello world test

- [SmolLM2-135M](https://hf-mirror.com/bartowski/SmolLM2-135M-Instruct-GGUF/tree/main)

<details>
<summary>Prompt format</summary>


```
<|im_start|>system
{system_prompt}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
```

</details>

In [ ]:
prefix = "<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\n"
suffix = "<|im_end|>\n<|im_start|>assistant\n"
# Q4_K_L == Q4_K_M, Q5_K_L == Q5_K_M
model = "models/SmolLM2-135M-Instruct-f16.gguf"
assert os.path.exists(model)

In [ ]:
question = "Give the most simple C program to print 'hello world!'" # Q4_K_S or under will be failed
prompt = prefix + question + suffix
!llama.cpp/bin/llama-cli -m {model} -p "{prompt}" -no-cnv -t 1 --temp 0 --top-k 0 --top-p 1.0 --min-p 0.0 2>/dev/null

In [ ]:
question = "Give the most simple C program to print 'Love world!'" # not work for Q8_0/Q6_K, Q5_K_S is the lowest, Q4 or under will be failed
# question = "Give the most simple C program to print 'love world!'" # Q4 or under will be failed
prompt = prefix + question + suffix
!llama.cpp/bin/llama-cli -m {model} -p "{prompt}" -no-cnv -t 1 --temp 0 --top-k 0 --top-p 1.0 --min-p 0.0 2>/dev/null

# Manga auther test

- [Llama-3.2-1B](https://hf-mirror.com/bartowski/Llama-3.2-1B-Instruct-GGUF/tree/main)
- [Official text_prompt_format](https://github.com/meta-llama/llama-models/blob/main/models/llama3_2/text_prompt_format.md)

<details>
<summary>Prompt format</summary>

```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
```


```python
# Q4_K_M/S 展现出了随着prefix和suffix微小变化的不稳定性
# prefix = "<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
# prefix = "<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Dec 2024\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
# prefix = "<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Oct 2024\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
# suffix = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>" # Q4_K_M only corrent for 火影忍者的作者是谁？ pair with Jul 2024
```

</details>

In [ ]:
import os
model_name = "Llama-3.2-1B-Instruct"
bf16_model = f"models/{model_name}-BF16.gguf"

add_residual = ""
# add_residual = "-IQ2_KS"

# quant_type = "IQ2_K" + add_residual
# quant_type = "IQ2_KS" + add_residual
# quant_type = "IQ2_XS" + add_residual

# quant_type = "exl3-2.0bpw" + add_residual
# quant_type = "exl3-2.25bpw" + add_residual
# quant_type = "exl3-2.5bpw" + add_residual

quant_type = "IQ4_XS"

model = f"models/{model_name}-{quant_type}.gguf"

assert os.path.exists(model)
prefix = "<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Aug 2024\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
suffix = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
model

<details>
<summary>perplexity results</summary>

```bash
wc -c < models/Llama-3.2-1B-Instruct-Q2_K.gguf  | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-205.49)/1856.2*16}'
wc -c < models/Llama-3.2-3B-Instruct-Q2_K.gguf  | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-308.23)/5376.6*16}'
wc -c < models/Meta-Llama-3.1-8B-Instruct-Q2_K.gguf  | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-410.98*2)/13313.0*16}'
```

`--output-tensor-type q6_K --token-embedding-type q6_K`


1B | ppl | +- | bpw | 3B | ppl | +- | bpw | 8B | ppl | +- | bpw
--- | --- | --- | --- | --- | ---| --- | --- | --- | --- | --- | ---
BF16* | 14.0027 | 0.10412 | 16 | | 10.5272 | 0.07553 | 16 | | 7.2549 | 0.04616 | 16
Q8_0 | 14.0230 | 0.10429 | 8.50 | | 10.5333 | 0.07562 | 8.50 | | 7.3325 | 0.04679 | 8.50
Q4_0*  | 16.2171 | 0.12277 | 4.50 | | 11.1097 | 0.08008 | 4.50 | | 7.7400 | 0.04970 | 4.50
Q4_K_M | 14.4839/14.4951 | 0.10791 | ~4.81 | | 10.7513/10.7890 | 0.07742 | ~4.79 | | 7.4842/7.4917 | 0.04741 | ~4.81
IQ4_KS | 14.5019 | 0.10764 | ~4.30 | | 10.7139 | 0.07692 | ~4.30 | | 7.4108 | 0.04714 | ~4.30
IQ4_XS | 14.6319 | 0.10877 | ~4.29 | | 10.7765 | 0.07772 | ~4.29 | | 7.4468 | 0.04755 | ~4.30
IQ4_KSS | 14.7195 | 0.10929 | ~4.06 | | 10.8588 | 0.07829 | ~4.06 | | 7.5270 | 0.04810 | ~4.06
exl3-3.0bpw | 15.3050 | 0.11378 | ~3.01 | | 11.2304 | 0.08075 | ~3.01 | | 7.7702 | 0.04939 | ~3.01
IQ3_XXS | 18.4877 | 0.13876 | ~3.09 | | 12.3744 | 0.08924 | ~3.05 | | 8.3654 | 0.05315 | ~3.09
IQ2_M | 23.7736 | 0.18104 | ~2.69 | | 14.2533 | 0.10413 | ~2.70 | | 9.4525 | 0.06126 | ~2.70
IQ2_S | 27.4562 | 0.21059 | ~2.47 | | 15.5285 | 0.11478 | ~2.48 | | 10.3411 | 0.06855 | ~2.48
| | |
IQ2_KS | 43.0943 | 0.34082 | ~2.25 | | 20.1127 | 0.14606 | ~2.24 | | 12.4773 | 0.08327 | ~2.25
IQ2_XS | 32.9898 | 0.25123 | ~2.36 | | 16.8900 | 0.12530 | ~2.36 | | 11.3635 | 0.07532 | ~2.37
IQ2_K | 26.1861 | 0.19856 | ~2.50 | | 14.5230 | 0.10375 | ~2.48 | | 10.1068 | 0.06623 | ~2.52
| | |
exl3-2.0bpw | 22.6485 | 0.16824 | ~2.01 | | 14.9236 | 0.10904 | ~2.01 | | 10.2131 | 0.06654 | ~2.01
exl3-2.25bpw | 19.8205 | 0.14798 | ~2.26 | | 13.7899 | 0.10146 | ~2.26 | | 9.3755 | 0.06035 | ~2.26
exl3-2.5bpw | 17.8907 | 0.13199 | ~2.51 | | 12.6631 | 0.09221 | ~2.51 | | 8.6809 | 0.05688 | ~2.51
| | |
IQ2_KS+IQ2_KS | 15.2246 | 0.11423 | | | 11.0023 | 0.07903 | | | 7.6160 | 0.04835
IQ2_K+IQ2_KS | 14.9130 | 0.10963 | | | 10.8116 | 0.07749 | | | 7.5286 | 0.04796
IQ2_XS+IQ2_KS | 14.9130 | 0.10963 | | | 10.9638 | 0.07862 | | | 7.6632 | 0.04898
exl3-2.0bpw+IQ2_KS | 15.2619 | 0.11412 | | | 10.8397 | 0.07685 | | | 7.6893 | 0.04919
exl3-2.25bpw+IQ2_KS | 14.7814 | 0.10981 | ? | | 10.9406 | 0.07897 | ??? | | 7.5638 | 0.04830
exl3-2.5bpw+IQ2_KS | 14.6485 | 0.10976 | | | 10.9590 | 0.07858 | | | 7.4580 | 0.04735
</details>

<details>
<summary>deprecated</summary>

1B | ppl | +- | bpw/size
--- | --- | --- | ---
exl3-2.0bpw+IQ2_KS | 14.8029 | 0.11115 |
exl3-2.25bpw+IQ2_KS | 14.7251 | 0.10991 |
IQ2_K+IQ2_KS | 14.4924 | 0.10771 | the best
IQ2_KS+IQ2_KS | 14.8918 | 0.11124 |
IQ2_XS+IQ2_KS | 14.9407 | 0.11150 |
    
</details>

<details>
<summary>perplexity results (legacy)</summary>

model | ppl | +- | bpw/size
--- | --- | --- | ---
[gptq-g32](https://huggingface.co/ModelCloud/Llama-3.2-1B-Instruct-gptqmodel-4bit-vortex-v2.5?show_file_info=model.safetensors) | 14.8567 | 0.11085 | 4@weight+(16@scale+4@zero)/32+32@g_dx/out_dim
[gptq-g128](https://hf-mirror.com/shuyuej/Llama-3.2-1B-Instruct-GPTQ?show_file_info=model.safetensors) | 17.8053 | 0.13670 | 4@weight+(16@scale+4@zero)/128+32@g_dx/out_dim
[awq](https://huggingface.co/AMead10/Llama-3.2-1B-Instruct-AWQ?show_file_info=model.safetensors)  | 15.2694 | 0.11399 | 4@weight+(16@scale+4@zero)/128
[IQ4_XS](https://hf-mirror.com/bartowski/Llama-3.2-1B-Instruct-GGUF?show_file_info=Llama-3.2-1B-Instruct-IQ4_XS.gguf) | 14.6856 | 0.10988 | 4.25 / 709MB
[Q4_0](https://hf-mirror.com/bartowski/Llama-3.2-1B-Instruct-GGUF?show_file_info=Llama-3.2-1B-Instruct-Q4_0.gguf) | 15.3289 | 0.11394 | 738MB
[Q4_K_S](https://hf-mirror.com/bartowski/Llama-3.2-1B-Instruct-GGUF?show_file_info=Llama-3.2-1B-Instruct-Q4_K_S.gguf) | 14.5792 | 0.10848 | 740MB
[Q5_K_S](https://hf-mirror.com/bartowski/Llama-3.2-1B-Instruct-GGUF?show_file_info=Llama-3.2-1B-Instruct-Q5_K_S.gguf) | 14.0893 | 0.10462 | 852MB
[Q5_K_M](https://hf-mirror.com/bartowski/Llama-3.2-1B-Instruct-GGUF?show_file_info=Llama-3.2-1B-Instruct-Q5_K_.gguf) | 14.0558 | 0.10458 | 870MB
llmc-awq-omniq | 14.0102 | 0.10419 | 4@weight+16@scale/128?
[w4a16g128asym](https://hf-mirror.com/numen-tech/Llama-3.2-1B-Instruct-w4a16g128asym) | 19.5997 | 0.14797 | 4@weight+16@scale/128
    
</details>



In [ ]:
# align with https://github.com/ikawrakow/ik_llama.cpp/discussions/63
!llama.cpp/bin/llama-perplexity -m {model} -f wikitext-2-raw/wiki.test.raw -t 1 -ngl 100

In [ ]:
# **Q5_K_M/Q5_K_S**/[Q4_K_M/Q4_K_S]/IQ4_XS will be failed
question = "Naruto的作者是谁？"
answer = "Naruto的作者是Masashi Kishimoto [end of text]"

In [ ]:
# [Q4_K_M/Q4_K_S]/IQ4_XS/awq/gptq will be failed
question = "火影忍者的作者是谁？"
answer = "火影忍者是由Masashi Kishimoto所创作的日本动画和漫画。 [end of text]"

In [ ]:
prompt = prefix + question + suffix
!llama.cpp/bin/llama-cli -m {model} -p "{prompt}" -c 2048 -n 512 --temp 0 --top-k 1 --seed 42 -ngl 100 2>/dev/null
print("\n"+answer)
model

## Q&A

In [ ]:
# Q4_0/awq will be failed
question = "Who is the author of 'Demon Slayer'?"
answer = "The author of the popular manga and anime series 'Demon Slayer: Kimetsu no Yaiba' is Koyoharu Gotoge. [end of text]"

In [ ]:
# awq will be failed
question = "Who is the author of 'Chainsaw Man'?"
answer = "The author of the manga and anime series 'Chainsaw Man' is Tatsuki Fujimoto. [end of text]"

In [ ]:
# [awq] will be failed
question = "Who is the author of 'Detective Conan'?"
answer = "The author of the popular manga and anime series 'Detective Conan' is Gosho Aoyama. [end of text]"

In [ ]:
# under-4-bit will be failed
question = "Who is the author of Manga 'Slam Dunk'?"
answer = 'The author of the popular manga series "Slam Dunk" is Takehiko Inoue. [end of text]'

In [ ]:
# under-4-bit will be failed
question = "Who is the author of 'Berserk'?"
answer = 'The author of the manga and anime series "Berserk" is Kentaro Miura. [end of text]'

# Original model file

In [ ]:
from safetensors import safe_open
model_path = "model.safetensors"

with safe_open(model_path, framework="pt") as f:
    tensor_names = f.keys()
    for name in tensor_names:
        tensor = f.get_tensor(name)
        print(f"name: {name}")
        print(f"shape: {tensor.shape}")
        print(f"type: {tensor.dtype}")
        print("-" * 50)

In [ ]:
with safe_open(model_path, framework="pt") as f:
  # tensor = f.get_tensor("model.norm.weight") # 'output_norm.weight'
  tensor = f.get_tensor("model.layers.0.input_layernorm.weight")
tensor

In [ ]:
import sys
hf_model = f"models/{model_name}"
bf16_model = f"models/{model_name}-BF16.gguf"
convert_hf_to_gguf_tool = f"{sys.prefix}/lib/python3.10/site-packages/convert_hf_to_gguf.py"
!python {convert_hf_to_gguf_tool} --outtype bf16 {hf_model} --outfile {bf16_model}

In [ ]:
from safetensors import safe_open
model_path = "model.safetensors"

with safe_open(model_path, framework="pt") as f:
    tensor_names = f.keys()
    byte_cnt = 0
    for name in tensor_names:
        if "model.layers" in name:
            tensor = f.get_tensor(name)
            byte_cnt += 2*tensor.numel()
byte_cnt/1024/1024

# imatrix and quantization

In [ ]:
# Get the imatrix file
calib_file = f"calibration_data/calibration_datav3.txt"
imatrix_file = f"models/{model_name}.imatrix"
!llama.cpp/bin/llama-imatrix -m {bf16_model} -f {calib_file} --output-file {imatrix_file} #-t 1 --chunks 1

In [ ]:
# Get the quant model
imatrix_file = f"models/{model_name}.imatrix"
!llama.cpp/bin/llama-quantize --output-tensor-type q6_K --token-embedding-type q6_K --imatrix {imatrix_file} {bf16_model} {model} {quant_type}
model

In [ ]:
# use gguf_dump.py to check quant_model
# the gguf_dump.py file is in (ik_)llama.cpp/scripts folder
import sys
gguf_dump_tool = f"{sys.prefix}/lib/python3.10/site-packages/scripts/gguf_dump.py"
!python {gguf_dump_tool} --markdown {model}

In [ ]:
# Given residual weight to quant it
draft_type = "IQ2_K"
residual_model = f"models/{model_name}-{draft_type}-residual.gguf"
# we don't care output-tensor and token-embedding, just don't want to quant them exactly
!llama.cpp/bin/llama-quantize --pure --output-tensor-type q6_K --token-embedding-type q6_K {residual_model} tmp.bin IQ2_KS

In [ ]:
target_model = f"models/{model_name}-IQ2_K-IQ2_KS.gguf"
draft_model = f"models/{model_name}-IQ2_K-dequant.gguf"

prefix = "<|start_header_id|>user<|end_header_id|>\n\n"
suffix = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

prompt = prefix+"Write 20 sentences about summer."+suffix
# prompt = prefix+"Who was the first prime minister of Britain?"+suffix
# prompt = prefix+"How many persons are needed to power a 800W toaster?"+suffix
# prompt = prefix+"Write the Quicksort algorithm in TypeScript."+suffix

!llama.cpp/bin/llama-speculative -m {target_model} -md {draft_model} \
-p "{prompt}" -c 2048 -n 512 --temp 0 --top-k 1 --seed 42 --draft-max 4 -ngl 100 -ngld 100

In [ ]:
!llama.cpp/bin/llama-cli -m {target_model} -p "{prompt}" -c 2048 -n 512 --temp 0 --top-k 1 --seed 42 -ngl 100 2>/dev/null
target_model

In [ ]:
!llama.cpp/bin/llama-cli -m {draft_model} -p "{prompt}" -c 2048 -n 512 --temp 0 --top-k 1 --seed 42 -ngl 100 2>/dev/null
draft_model